In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from Utils.Postprocessing.post_process import postprocess_mask, merge_nearest_endocards
from Utils.Postprocessing.create_geojson import create_geojson

import geojson

In [ ]:
def display_sample(display_list, name, title=None, legend=False,):
    print(legend)
    plt.figure(figsize=(18, 18))

    for i in range(len(display_list)):
        ax = plt.subplot(1, len(display_list), i+1)
        
        if legend:
            black_patch = mpatches.Patch(color='black', label='Pozadie')
            red_patch = mpatches.Patch(color='red', label='Skutočná maska')
            green_patch = mpatches.Patch(color='green', label='Predikcia')
            yellow_patch = mpatches.Patch(color='yellow', label='Správna predikcia')


            ax.legend(handles=[black_patch, red_patch, green_patch, yellow_patch])
        
        if title:
            plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    if len(title) == 1:
        plt.savefig(f"{name} - {title[0]}.svg")
    plt.show()

In [ ]:
def get_labels():
    return np.asarray(
        [
            [255, 255, 255], # background
            [128, 0, 0], #blood_vessels
            [0, 128, 0], # inflammations
            [128, 128, 0], # endocard
            [0, 0, 128],
            [128, 0, 128],
            [0, 128, 128],
            [0, 255, 192],
            [255, 170, 0],
            [192, 128, 255]
        ]
    )

In [ ]:
def convert_to_multiclass(mask, num_classes):
    new_mask = np.zeros((mask.shape[0], mask.shape[1], num_classes), dtype=int)
    
    new_mask[:, :, 4] = mask[:, :, 0]
    new_mask[:, :, 5] = mask[:, :, 1]
    new_mask[:, :, 6] = mask[:, :, 2]
    
    new_mask[:, :, 1] = cv.bitwise_and(new_mask[:, :, 4], new_mask[:, :, 5])    # blood + infla
    new_mask[:, :, 2] = cv.bitwise_and(new_mask[:, :, 6], new_mask[:, :, 5])    # endocard + infla
    new_mask[:, :, 3] = cv.bitwise_and(new_mask[:, :, 4], new_mask[:, :, 6])    # blood + endocard
    
    union = new_mask[:, :, 0]
    for i in range(1, num_classes):
        union = cv.bitwise_or(union, new_mask[:, :, i])
    union = np.clip(union, 0, 1)
    new_mask[:, :, 0] = np.where((union == 0) | (union == 1), union ^ 1, union)

    new_mask[:, :, 4] = cv.bitwise_xor(new_mask[:, :, 4], new_mask[:, :, 1])
    new_mask[:, :, 4] = cv.bitwise_xor(new_mask[:, :, 4], new_mask[:, :, 3])
    
    new_mask[:, :, 5] = cv.bitwise_xor(new_mask[:, :, 5], new_mask[:, :, 1])
    new_mask[:, :, 5] = cv.bitwise_xor(new_mask[:, :, 5], new_mask[:, :, 2])
    
    new_mask[:, :, 6] = cv.bitwise_xor(new_mask[:, :, 6], new_mask[:, :, 2])
    new_mask[:, :, 6] = cv.bitwise_xor(new_mask[:, :, 6], new_mask[:, :, 3])

    return new_mask

In [ ]:
def create_RGB_mask(mask, labels, shape):
    canvas = np.zeros((shape[0], shape[1], 3))
    for x in range(shape[0]):
        for y in range(shape[1]):
            canvas[x, y, :] = labels[mask[x, y]]

    return canvas

In [ ]:
def get_mask_index(feature, classes):
    class_type = feature['properties']['classification']['name']

    for idx, name in enumerate(classes):
        if class_type.lower() == name.lower():
            return idx

    # else return Other cells
    return 0


def get_mask(shape, annotations, classes):
    x, y = int(shape[0]), int(shape[1])

    classes_masks = [
        np.zeros((x, y, 1), dtype='uint8')
        for _ in range(len(classes))
    ]

    for feat in annotations:
        geometry_name = 'geometry'
        coors = feat[geometry_name]['coordinates'][0]
        pts = [[round(c[0]), round(c[1])] for c in coors]
        cv.fillPoly(
            classes_masks[get_mask_index(feat, classes)],
            [np.array(pts)],
            1
        )  # fill with ones if cell present

    mask = np.concatenate(classes_masks, axis=2)
    return mask

In [ ]:
def create_prediction_img(true, pred, frag_x, frag_y, tissue=None):
    new_image = np.zeros((pred.shape[0], pred.shape[1], 3))
    new_image[:, :, 0] = true
    new_image[:, :, 1] = pred

    #new_image[np.where((new_image==[0, 0, 0]).all(axis=2))] = [1,1,1]

    if tissue:
        color = (0, 0, 0)
        thickness = 2
        pts = [[round(c[0]) - frag_x, round(c[1]) - frag_y] for c in tissue[0]]
        new_image = cv.polylines(new_image, [np.array(pts)], True, color, thickness)

    return new_image

In [ ]:
def vizualize(data_path, ground_truth_path, predicted_path, tissue_mask, shape, num_classes, fragment):
    classes = ["Endocarium"]#["Blood vessels", "Inflammation", "Endocarium"]
    frag_x, frag_y, width, height = fragment

    name = predicted_path.split('\\')[-2]
    print(name)
    
    
    img = np.load(data_path)
    gj = geojson.load(open(ground_truth_path))
    ground_truth = get_mask(shape, gj['features'], classes)
    print('Mask Created!')
    ground_truth = ground_truth[frag_y:frag_y + height, frag_x:frag_x + width]
    
    gj = geojson.load(open(predicted_path))
    predicted = get_mask(shape, gj['features'], classes)
    #predicted = postprocess_mask(predicted, dilate=True)
    print(predicted.shape)
    #predicted = merge_nearest_endocards(predicted)
    print(predicted.shape)
    
    print('Mask Created!')
    predicted = predicted[frag_y:frag_y + height, frag_x:frag_x + width]

    gj = geojson.load(open(tissue_mask))
    #tissue = gj[0]['geometry']['coordinates'][1]
    tissue = None
    
    #geo_file = create_geojson(predicted, ['blood_vessels', 'inflammations', 'endocariums'])
    
    
    #print('Mask Converted!')
    #x, y, size = 3500, 1500, 2000
    #x, y, size = 6500, 1500, 2000
    #x, y, size = 500, 500, 2000
    x, y, size = 0, 0, 100_000
    
    display_sample([img[y:y + size,x:x + size]], name, ['Originálny snímok', 'Pôvodná maska', 'Predikcia'])
    display_sample([create_prediction_img(ground_truth[y:y + size,x:x + size, 0], predicted[y:y + size,x:x + size, 0], frag_x, frag_y)], name, [classes[0]], legend=False)
    #display_sample([create_prediction_img(ground_truth[y:y + size,x:x + size, 1], predicted[y:y + size,x:x + size, 1], frag_x, frag_y)], name, [classes[1]], legend=False)
    #display_sample([create_prediction_img(ground_truth[y:y + size,x:x + size, 2], predicted[y:y + size,x:x + size, 2], frag_x, frag_y)], name, [classes[2]], legend=False)

In [ ]:
vizualize(
    data_path=r'D:\Master Thesis\Code\Segmentation\data4\images\7026_21_HE_1_0.npy',
    ground_truth_path=r'D:\Master Thesis\Data\Annotations\7026_21_HE.vsi - 20x.geojson',
    predicted_path=r'D:\Master Thesis\DP3 results\Evaluation 11-04-2023\U-Net++\7026_21_HE.geojson',
    tissue_mask=r'D:\Master Thesis\Data\EMB-IKEM-2022-03-09\7026_21_HE.vsi - 20x_tissue and cells.geojson',
    shape=(9849, 33381),
    num_classes=7,
    fragment=(9671, 2839, 8600, 5192)
)

In [ ]:
#endokard
vizualize(
    data_path=r'D:\Master Thesis\Code\Segmentation\data5\SRel images\3002_21_SRel_4_0.npy',
    ground_truth_path=r'D:\Master Thesis\Data\SRel annotations\3002_21_SRel.vsi - 20x_annotations.geojson',
    predicted_path=r'E:\Master Thesis\DP3 results\SRel2\3002_21_SRel.geojson',
    tissue_mask=r'D:\Master Thesis\Data\EMB-IKEM-2022-03-09\QuPath project EMB - anotations\SRel classification\3002_21_SRel.vsi - 20x.geojson',
    shape=(14374, 43593),
    num_classes=7,
    fragment=(32199, 1023, 10616, 6648)
)

In [ ]:
plt.figure(figsize=(10, 10))
black_patch = mpatches.Patch(color='black', label='Pozadie')
red_patch = mpatches.Patch(color='red', label='Skutočná maska')
green_patch = mpatches.Patch(color='green', label='Predikcia')
yellow_patch = mpatches.Patch(color='yellow', label='Správna predikcia')
ax = plt.subplot(1, 1, 1)
ax.legend(handles=[black_patch, red_patch, green_patch, yellow_patch], fontsize = 25)
plt.show()

In [ ]:
def process_image(mask, dilate=False):
    kernel_size = (51, 51)
    
    for idx in range(mask.shape[2]):
        img_modified = mask[:, :, idx]

        img_modified = cv.morphologyEx(img_modified, cv.MORPH_OPEN, kernel_size)

        horizontal_kernel = cv.getStructuringElement(cv.MORPH_RECT, (75, 5))
        detected_lines = cv.morphologyEx(img_modified, cv.MORPH_OPEN, horizontal_kernel, iterations=2)
        cnts = cv.findContours(detected_lines, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            cv.drawContours(img_modified, [c], -1, (0, 0, 0), 5)
        
        contours, _ = cv.findContours(img_modified, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if len(cnt) < 5:
                cv.fillPoly(
                    img_modified,
                    [cnt],
                    0
                )

        img_modified = cv.GaussianBlur(img_modified, kernel_size, 0)
        
        contours, _ = cv.findContours(img_modified, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv.contourArea(cnt)
            #print(cnt, len(cnt))
            if area < 2500:
                cv.fillPoly(
                    img_modified,
                    [cnt],
                    0
                )

        img_modified = cv.morphologyEx(img_modified, cv.MORPH_OPEN, kernel_size * 5)
        if dilate:
            img_modified = cv.dilate(img_modified, kernel_size * 5, 0)
        
        mask[:, :, idx] = img_modified
        
    return mask

In [ ]:
def experiment(data_path, ground_truth_path, predicted_path, shape, num_classes, fragment):
    classes = ["Blood vessels", "Inflammation", "Endocarium"]
    x, y, width, height = fragment
    
    
    img = np.load(data_path)
    gj = geojson.load(open(ground_truth_path))
    ground_truth = get_mask(shape, gj['features'], classes)
    print('Mask Created!')
    ground_truth = ground_truth[y:y + height, x:x + width]
    ground_truth_class = convert_to_multiclass(ground_truth, num_classes)
    
    ground_truth_class = np.argmax(ground_truth_class, axis=-1)
    ground_truth_class = np.expand_dims(ground_truth_class, -1)
    print('Mask Converted!')
    
    gj = geojson.load(open(predicted_path))
    predicted = get_mask(shape, gj['features'], classes)
    print('Mask Created!')
    predicted = predicted[y:y + height, x:x + width]
    
    print('Mask Converted!')
    #x, y, size = 3500, 1500, 2000
    #x, y, size = 6500, 1500, 2000
    #x, y, size = 500, 500, 2000
    x, y, size = 0, 0, 100_000
    
    # predicted = process_image(predicted, dilate=True)
    
    display_sample([img[y:y + size,x:x + size]], ['Originálny snímok', 'Pôvodná maska', 'Predikcia'])
    
    display_sample([create_prediction_img(ground_truth[y:y + size,x:x + size, 0], predicted[y:y + size,x:x + size, 0])], [classes[0]], legend=False)
    display_sample([create_prediction_img(ground_truth[y:y + size,x:x + size, 1], predicted[y:y + size,x:x + size, 1])], [classes[1]], legend=False)
    display_sample([create_prediction_img(ground_truth[y:y + size,x:x + size, 2], predicted[y:y + size,x:x + size, 2])], [classes[2]], legend=False)

In [ ]:
experiment(
    data_path=r'D:\Master Thesis\Code\Segmentation\data4\images\7026_21_HE_1_0.npy',
    ground_truth_path=r'D:\Master Thesis\Data\Annotations\7026_21_HE.vsi - 20x.geojson',
    predicted_path=r'D:\Master Thesis\Nested U-Net\7026_21_HE.geojson',
    shape=(9849, 33381),
    num_classes=7,
    fragment=(9671, 2839, 8600, 5192)
)